# Analyze problem

* the problem is expected in pragmatic format

In [18]:
# run this to install dependencies
#!{sys.executable} -m pip install pandas matplotlib ipyleaflet

import json
import pandas as pd
import matplotlib.pyplot as plt

# change to your problem
problem_path = "../experiments/demo/problems/simple.multi-job.25.json"

# change amount of rows displayed by data frame
# pd.set_option('display.max_rows', None)

with open(problem_path) as problem_file:
    problem = json.load(problem_file)

## Plan

In [19]:
plan = problem['plan']
jobs = pd.json_normalize(plan, record_path=['jobs'])

# show summary of jobs
jobs_summary = jobs.apply(lambda job: pd.concat([
    pd.Series([job['id'], len(job['pickups']), len(job['deliveries'])], index=['id', 'pickups', 'deliveries']),
    job.filter(regex = 'skills*')]), axis=1
)


In [20]:
# get size of pickups and deliveries

pd.concat([jobs_summary.groupby('pickups').count(), jobs_summary.groupby('deliveries').count()], axis=1).drop(list(jobs_summary.filter(regex='id|skills*')), axis=1)

,deliveries,pickups
1,15,15
3,10,10


In [21]:
# get pickup demand

jobs.apply(
    lambda job: pd.json_normalize(job['pickups'])['demand'],
    axis=1
)

,0,1,2
0,[3],NaN,NaN
1,[1],[1],[1]
2,[1],[1],[1]
3,[3],NaN,NaN
4,[1],NaN,NaN
5,[1],[1],[1]
6,[1],[1],[1]
7,[1],[1],[1]
8,[1],[1],[1]
9,[1],NaN,NaN


In [22]:
# get delivery demand

jobs.apply(
    lambda job: pd.json_normalize(job['deliveries'])['demand'],
    axis=1
)

,0,1,2
0,[1],[1],[1]
1,[3],NaN,NaN
2,[3],NaN,NaN
3,[1],[1],[1]
4,[1],NaN,NaN
5,[3],NaN,NaN
6,[3],NaN,NaN
7,[3],NaN,NaN
8,[3],NaN,NaN
9,[1],NaN,NaN


In [23]:
# group subjobs by the same location

subjob_type='deliveries'
subjob_idx=0
place_idx=0

def loc_to_key(loc):
    return "{},{}".format(loc['lat'], loc['lng'])

pickup_locations = jobs.apply(
    lambda job: pd.Series({'loc': loc_to_key(job[subjob_type][subjob_idx]['places'][place_idx]['location']), 'count': 1 }),
    axis=1
)

pickup_locations.groupby('loc').count()


,count
loc,
"52.46511694386838,13.297678183052604",1
"52.47024351434404,13.367425628544071",1
"52.47158077561322,13.428073908706715",1
"52.47402861212452,13.511200299551914",1
"52.47592514299791,13.363600724841861",1
"52.476201585973165,13.465323704730976",1
"52.48294772163354,13.506479369983362",1
"52.494441122410066,13.310976910089025",1
"52.500197154330266,13.420403735758104",1


# Fleet

In [24]:
# get vehicle summary

fleet = problem['fleet']
vehicles = pd.json_normalize(fleet, record_path=['vehicles'])


def get_shift_time(vehicle):
    shift = vehicle['shifts'][0]
    shift_time = shift['start']['earliest']
    if 'end' in shift:
        return "{} - {}".format(shift_time, shift['end']['latest'])
    else:
        return shift_time

vehicle_summary = vehicles.apply(lambda vehicle: pd.concat([
    pd.Series(
        [vehicle['typeId'], len(vehicle['vehicleIds']), len(vehicle['shifts']), get_shift_time(vehicle), vehicle['capacity']], 
        index=['typeId', 'amount', 'shifts', 'shift time [0]', 'capacity']),
    vehicle.filter(regex = 'skills*')]), axis=1
)

vehicle_summary

,typeId,amount,shifts,shift time [0],capacity
0,vehicle,5,1,2020-05-01T09:00:00.00Z - 2020-05-01T18:00:00.00Z,[20]


In [25]:
# get shift start location distribution
shift_idx=0

def loc_to_key(loc):
    return "{},{}".format(loc['lat'], loc['lng'])

shift_locations = vehicles.apply(
    lambda vehicle: pd.Series({'loc': loc_to_key(vehicle['shifts'][shift_idx]['start']['location']), 'count': 1 }),
    axis=1
)

shift_locations.groupby('loc').count()

,count
loc,
"52.4181,13.4637",1


In [26]:
# get shift start location distribution
shift_idx=0

shift_locations = vehicles.apply(
    lambda vehicle: pd.Series({'loc': vehicle['shifts'][shift_idx]['start']['earliest'], 'count': 1 }),
    axis=1
)

shift_locations.groupby('loc').count()

,count
loc,
2020-05-01T09:00:00.00Z,1


# Visualize on map

## get all jobs locations

In [27]:
def extract_subjobs_locations(subjobs):
    locations = []
    for subjob in subjobs:
        for place in subjob['places']:
            locations.append(place['location'])
    
    return locations

def extract_all_job_locations(job):
    # TODO make sure no conflicts here
    subjob_types = [{ 'key': 'pickups', 'type': 'pickup' }, { 'key': 'deliveries', 'type': 'delivery' }]
    locations_map = {}

    for subjob_type in subjob_types:
        if subjob_type['key'] in job:
            locations_map[subjob_type['type']] = extract_subjobs_locations(job[subjob_type['key']])
    
    return locations_map


jobs_locations = jobs.apply(lambda job: extract_all_job_locations(job), axis=1)


## get all vehicles locations

In [28]:
# get vehicle locations
def extract_all_vehicle_locations(vehicle):
    locations_map = {
        'start' : [],
        'end' : [],
        'break': []
    }

    for shift in vehicle['shifts']:
        locations_map['start'].append(shift['start']['location'])

        if 'end' in shift:
            locations_map['end'].append(shift['end']['location'])

        if 'breaks' in shift:
            for br in shift['breaks']:
                if 'location' in br:
                    locations_map['break'].append(br['location'])

    return locations_map


vehicles_locations = vehicles.apply(lambda vehicle: extract_all_vehicle_locations(vehicle), axis=1)

## create geojson

In [29]:
geojson_data = {
    'type': 'FeatureCollection',
    'features': []
}

all_locations = []

# NOTE would be better to get bounds automatically from ipyleaflet
def get_bounds(locations):
    min_lat = 200
    min_lon = 200
    max_lat = -200
    max_lon = -200

    for location in locations:
        if min_lat > location[1]:
            min_lat = location[1]

        if min_lon > location[0]:
            min_lon = location[0]
        
        if max_lat < location[1]:
            max_lat = location[1]
        
        if max_lon < location[0]:
            max_lon = location[0]

    return ((min_lat, min_lon), (max_lat, max_lon))

def create_point(style_type, location):
    return {
        'type': 'Feature',
        'properties': {
            'type': style_type
        },
        'geometry': {
            'type': 'Point',
            'coordinates': location
        }
    }

def add_locations(style_type, location):
    for location in locations:
        location = location['lng'], location['lat']
        all_locations.append(location)
        geojson_data['features'].append(create_point(style_type, location))

for job_location_map in jobs_locations:
    for style_type, locations in job_location_map.items():
        add_locations(style_type, locations)

for vehicle_location_map in vehicles_locations:
    for style_type, locations in vehicle_location_map.items():
        add_locations(style_type, locations)

bounds = get_bounds(all_locations)

## visualize geojson

In [30]:
from ipyleaflet import Map, GeoJSON

def apply_style(feature):
    style_type = feature['properties']['type']

    if style_type == 'pickup':
        return {'color': 'red', 'fillColor': 'blue'}

    if style_type == 'delivery':
        return {'color': 'blue', 'fillColor': 'purple'}
    
    if style_type == 'start':
        return {'color': 'white', 'fillColor': 'black'}

    if style_type == 'end':
        return {'color': 'black', 'fillColor': 'white', 'radius': 10}

    return {'color': 'yellow', 'fillColor': 'green'}

m = Map(center=(0, 0), zoom=3)

geojson_layer = GeoJSON(
    data=geojson_data,
    point_style={'radius': 5, 'weight': 3, 'fillOpacity': 0.5 },
    style_callback=apply_style,
)

m.add_layer(geojson_layer)
m.fit_bounds(bounds)

m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…